![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/Spark_NLP_Udemy_MOOC/Healthcare_NLP/Mapper2Chunk.ipynb)

# **📜 Mapper2Chunk**

This notebook will cover the different parameters and usages of `Mapper2Chunk`.

**📖 Learning Objectives:**

1. Understand how this annotator converts 'LABELED_DEPENDENCY' type annotations
into 'CHUNK' type.

2. Learn how to create a new chunk-type column compatible with annotators that use chunk type as input.

3. Customize your chunk-type annotations by using the different parameters of the annotator.

**🔗 Helpful Links:**

- Reference Documentation: [Mapper2Chunk](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#mapper2chunk)



## **🎬 Colab Setup**

In [ ]:
# Install the johnsnowlabs library to access Spark-NLP for Healthcare
! pip install -q johnsnowlabs

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import nlp, medical

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.settings.enforce_versions=False
nlp.install(refresh_install=True)

👌 Detected license file /content/license_keys.json
🚨 Outdated Medical Secrets in license file. Version=5.4.0.PR but should be Version=5.4.0
🚨 Outdated OCR Secrets in license file. Version=5.3.2 but should be Version=5.4.0
📋 Stored John Snow Labs License in /root/.johnsnowlabs/licenses/license_number_0_for_Spark-Healthcare_Spark-OCR.json
👷 Setting up  John Snow Labs home in /root/.johnsnowlabs, this might take a few minutes.
🙆 JSL Home setup in /root/.johnsnowlabs
Running "/usr/bin/python3 -m pip install https://pypi.johnsnowlabs.com/[LIB_SECRET]/spark-nlp-jsl/spark_nlp_jsl-5.4.0-py3-none-any.whl --force-reinstall"
Installed 1 products:
💊 Spark-Healthcare==5.4.0 installed! ✅ Heal the planet with NLP! 


In [ ]:
# Automatically load license data and start a session with all jars user has access to
spark = nlp.start()

👌 Detected license file /content/license_keys.json
👌 Launched cpu optimized session with with: 🚀Spark-NLP==5.4.0, 💊Spark-Healthcare==5.4.0, running on ⚡ PySpark==3.4.0


In [ ]:
spark

## **🖨️ Input/Output Annotation Types**

- Input: `LABELED_DEPENDENCY`

- Output: `CHUNK`

## **🔎 Parameters**

**Parameters**:

- **inputCols**: Input annotation columns, typically `["relations"]`, containing `LABELED_DEPENDENCY` type annotations from `ChunkMapper`.

- **outputCol**: Name of the output column that will store the resulting `CHUNK` annotations.

- **filterNoneValues**: Whether to filter out `NONE` or empty values when converting labeled dependencies to chunks. Default: `False`.



### Pipeline

In [ ]:
text = "Patient resting in bed. Patient given azithromycin without any difficulty. Patient denies nausea at this time. Zofran declined. Patient is also having intermittent sweating"
data = spark.createDataFrame([[text]]).toDF("text")

In [ ]:
documentAssembler = nlp.DocumentAssembler() \
  .setInputCol("text") \
  .setOutputCol("document")

sentenceDetector = nlp.SentenceDetector() \
  .setInputCols(["document"]) \
  .setOutputCol("sentence")

tokenizer = nlp.Tokenizer() \
  .setInputCols(["sentence"]) \
  .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models") \
  .setInputCols(["sentence", "token"]) \
  .setOutputCol("embeddings")

clinical_ner = medical.NerModel.pretrained("ner_jsl", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")

ner_converter = medical.NerConverterInternal() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")

chunkMapper = medical.ChunkMapperModel.pretrained("drug_action_treatment_mapper", "en", "clinical/models") \
  .setInputCols(["ner_chunk"]) \
  .setOutputCol("relations") \
  .setRels(["action"])

pipeline = nlp.Pipeline(
    stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer,
      word_embeddings,
      clinical_ner,
      ner_converter,
      chunkMapper
])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_jsl download started this may take some time.
[OK!]
drug_action_treatment_mapper download started this may take some time.
[OK!]


In [ ]:
result = pipeline.fit(data).transform(data)

In [ ]:
result.selectExpr("relations.result", "relations.annotatorType").show(truncate=False)

+-------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|result                                                 |annotatorType                                                                                       |
+-------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|[bactericidal, antiemetic, anti-abstinence, NONE, NONE]|[labeled_dependency, labeled_dependency, labeled_dependency, labeled_dependency, labeled_dependency]|
+-------------------------------------------------------+----------------------------------------------------------------------------------------------------+



### `setFilterNoneValues`

In [ ]:
mapper2chunk = medical.Mapper2Chunk() \
  .setInputCols(["relations"]) \
  .setOutputCol("chunk") \
  .setFilterNoneValues(True)

pipeline = nlp.Pipeline(
    stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer,
      word_embeddings,
      clinical_ner,
      ner_converter,
      chunkMapper,
      mapper2chunk
])

In [ ]:
result = pipeline.fit(data).transform(data)

In [ ]:
result.selectExpr("chunk.result", "chunk.annotatorType").show(truncate=False)

+-------------------------------------------+---------------------+
|result                                     |annotatorType        |
+-------------------------------------------+---------------------+
|[bactericidal, antiemetic, anti-abstinence]|[chunk, chunk, chunk]|
+-------------------------------------------+---------------------+

